In [1]:
import re
import json
import numpy as np
import pandas as pd
import jieba
from zhconv import convert
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA

In [3]:
# 读取8月份的数据
cvjd_202208_6108 = pd.read_pickle('../generalization_data/cvjd_202208_6108.pkl').reset_index(drop=True)
cvjd_202208_6108

,cv_id,jd_id,da_label,rank_label,resume_type,order_month
0,8a69c41582926ae60182a07b24c1048c,8a69f62f82926a8601829f1379a07648,2,2,4.0,2022-08
1,8a69c41582926ae60182a0e09bf27664,8a69d77e82926b410182a083fbf602c0,3,3,4.0,2022-08
2,8a69c41582926ae60182a119de713808,8a69f68782590ecb01826326b5e94919,-1,2,4.0,2022-08
3,8a69c41582a3cc240182b0354c0e7bad,2c92071581ebf3390181ec9e500755e4,1,1,4.0,2022-08
4,8a69c4a3827cc886018284b9612d0f03,8a69f601823a92b0018243dcb9c45f7a,-1,1,4.0,2022-08
...,...,...,...,...,...,...
6103,8a69d71b82add4e10182aebff7d016d2,8a69c468826ee9b4018290a64a0c01bb,2,2,4.0,2022-08
6104,8a69d7b4826d7dfb01826d7efad200f1,2c920715818a462a01818e3f9c0a2337,3,3,4.0,2022-08
6105,8a69d7c28219945301823a12e0266463,8a69c4df82b081d30182c9ec1ecd1c67,-1,1,5.0,2022-08
6106,8a69d7c282199453018247bb00cb68a3,8a69c468826ee9b40182831bedfc5e79,3,3,4.0,2022-08


In [5]:
cvjd_202208_6108['order_month'].value_counts()

2022-08    6108
Name: order_month, dtype: int64

In [4]:
# 这里是做一个检查。训练集和测试集的数据截止到20220731
cvjd_traintest_73456 = pd.read_pickle('../generalization_data/cvjd_traintest_73456.pkl').reset_index(drop=True)
cvjd_traintest_73456

,cv_id,jd_id,da_label,rank_label,resume_type,order_month
0,2c9207157bcb9549017bce3ea49b00fa,2c9207157bcbb76b017bce3d99ba059a,1,1,NaN,2021-09
1,2c9207157bf91042017c1040330d04d3,2c9207157bcbb76b017bec76efca1ca5,-1,1,4.0,2021-09
2,2c9207157bf91042017c10f871d309f4,2c9207157f347800017f624770866105,1,1,4.0,2022-03
3,2c9207157bf91042017c62fce2583411,2c9207157b9b9199017bca11fd3d3f11,2,2,NaN,2021-10
4,2c9207157bf91042017c6ecc163d3aa8,2c9207157bf94d99017c6ecb294930b1,3,3,NaN,2021-10
...,...,...,...,...,...,...
73451,8a69d7a68219933d01823d93f8022296,2c92071581ebf3390182009a771b5b07,1,1,4.0,2022-07
73452,8a69d7c2821994530182334ac6bb128e,8a69c4a8820f0ee00182143af21d26f8,-1,2,4.0,2022-07
73453,8a69d7c2821994530182386fdad44a80,8a69f67f82198d89018223bca0291137,3,3,4.0,2022-07
73454,8a69d7c282199453018246ebf6b60643,8a69f67f82198d89018234adfcd91510,-1,2,4.0,2022-07


In [6]:
cvjd_traintest_73456['order_month'].value_counts()

2022-04    15247
2022-06    13917
2022-05    12662
2022-03    10293
2022-02     7969
2022-07     7677
2022-01     3210
2021-12     1748
2021-11      328
2021-09      264
2021-10      141
Name: order_month, dtype: int64